In [ ]:
import os, inspect, sys, pathlib, ipynbname, glob, shutil
import ansys.edb

from ansys.edb.database import *
from ansys.edb.definition import *
from ansys.edb.geometry import *
from ansys.edb.hierarchy import *
from ansys.edb.layer import *
from ansys.edb.layout import *
from ansys.edb.layout_instance import *
from ansys.edb.net import *
from ansys.edb.primitive import *
from ansys.edb.simulation_setup import *
from ansys.edb.terminal import *
from ansys.edb.database import Database
from ansys.edb.session import launch_session

In [ ]:
def get_fixture_file(filename):
    example_name = ipynbname.name()
    return os.path.join(os.getcwd(), 'fixtures', example_name, filename)
    
def get_input_filename():
    example_name = ipynbname.name()
    return os.path.join(os.getcwd(), f'{example_name}.aedb')
    
def get_output_filename():
    example_name = ipynbname.name()
    return os.path.join(get_output_dir(), f'{example_name}.aedb')

def get_output_files():
    example_name = ipynbname.name()
    return os.path.join(get_output_dir(), f'{example_name}.*')

def get_output_dir():
    return os.path.join(os.getcwd(), "results")


In [ ]:
%run helpers/example_teardown.ipynb

RPC_SERVER = None
session = launch_session(RPC_SERVER, 50051)

In [ ]:
if not pathlib.Path(get_output_dir()).is_dir():
    os.makedirs(get_output_dir())

for f in glob.glob(get_output_files()):
    if pathlib.Path(f).is_dir():
        shutil.rmtree(f)
    else:
        os.remove(f)
    
if pathlib.Path(get_input_filename()).is_file():
    db = Database.open(get_input_filename())
    cell = db.cells()[0]
else:
    db = Database.create(get_output_filename())
    cell = Cell.create(db, CellType.CIRCUIT_CELL, "Example")

layout = cell.layout
gnd_net = Net.create(layout, 'GND')

print('defined db, cell, layout, gnd_net variables.')